In [1]:
from logging import raiseExceptions
from tokenize import Double
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import meshplot as mp
import torch
torch.manual_seed(125)
import random
random.seed(125)
import torch_f as torch_f
from modelovae import Node, GRASSEncoder, GRASSDecoder, traverseFeatures, deserialize
from meshSubplot import sTree, plotTree

In [2]:
use_gpu = True
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")

def count_fn(f):
    def wrapper(*args, **kwargs):
        wrapper.count += 1
        return f(*args, **kwargs)
    wrapper.count = 0
    return wrapper

In [3]:
def traverse(root, tree):
       
        if root is not None:
            traverse(root.left, tree)
            tree.append((root.radius, root.data))
            traverse(root.right, tree)
            return tree

def traversem(root):
       
        if root is not None:
            traversem(root.left)
            print(root)
            traversem(root.right)
            return 

def traverse_2(tree1, tree2, t_l):
       
        if tree1 is not None:
            traverse_2(tree1.left, tree2.left, t_l)
            if tree2:
                t_l.append((tree1.radius, tree2.radius))
                print((tree1.radius, tree2.radius))
            else:
                t_l.append(tree1.radius)
                print((tree1.radius))
            traverse_2(tree1.right, tree2, t_l)
            return t_l
            

def traverse_conexiones(root, tree):
        """
        traverse function will print all the node in the tree.
        """
        if root is not None:
            traverse_conexiones(root.left, tree)
            if root.right is not None:
                tree.append((root.data, root.right.data))
            if root.left is not None:
                tree.append((root.data, root.left.data))
            traverse_conexiones(root.right, tree)
            return tree


@count_fn
def createNode(data, radius, position = None, left = None, right = None, cl_prob = None, ce = None, mse=None):
        """
        Utility function to create a node.
        """
        return Node(data, radius, left, right)


def read_tree(filename, dir):
    #with open('./prof6/' +filename, "r") as f:
    #with open('./trees/' +filename, "r") as f:
    #with open('./' +dir +'/' +filename, "r") as f:
    with open(dir +'/' +filename, "r") as f:
        byte = f.read() 
        return byte

In [4]:
def encode_structure_fold(fold, root):
    
    
    def encode_node(node):
        
        if node is None:
            return
        
        if node.isLeaf():
            return fold.add('leafEncoder', node.radius)
        else:
            left = encode_node(node.left)
            right = encode_node(node.right)
            if left is not None and right is not None:
                return fold.add('bifurcationEncoder', node.radius, right, left)
            elif right is not None:
                return fold.add('internalEncoder', node.radius, right)
            elif left is not None:
                return fold.add('internalEncoder', node.radius, left)
        

    encoding = encode_node(root)
    
    #return encoding
    return fold.add('sampleEncoder', encoding)

def encode_structure(root, Grassencoder):
        
    def encode_node(node, Grassencoder):
          
        if node is None:
            return
        if node.is_leaf():
            return Grassencoder.leafEncoder(node.radius.reshape(-1,4))
        else :
            left = encode_node(node.left, Grassencoder)
            right = encode_node(node.right, Grassencoder)
            if left is not None and right is not None:
                return Grassencoder.bifurcationEncoder(node.radius.reshape(-1,4), right, left)
            if right is not None:
                return Grassencoder.internalEncoder(node.radius.reshape(-1,4), right)
            if left is not None:
                return Grassencoder.internalEncoder(node.radius.reshape(-1,4), left)

    encoding = encode_node(root, Grassencoder)
    return encoding


In [5]:
def numerar_nodos(root, count):
    if root is not None:
        numerar_nodos(root.left, count)
        root.data = len(count)
        count.append(1)
        numerar_nodos(root.right, count)
        return 


def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def norm(root, minx, miny, minz, minr, maxx, maxy, maxz, maxr):
    
    if root is not None:
        mx = minx.clone().detach()
        my = miny.clone().detach()
        mz = minz.clone().detach()
        mr = minr.clone().detach()
        Mx = maxx.clone().detach()
        My = maxy.clone().detach()
        Mz = maxz.clone().detach()
        Mr = maxr.clone().detach()
       
        root.radius[0] = (root.radius[0] - minx)/(maxx - minx)
        root.radius[1] = (root.radius[1] - miny)/(maxy - miny)
        root.radius[2] = (root.radius[2] - minz)/(maxz - minz)
        root.radius[3] = (root.radius[3] - minr)/(maxr - minr)
        
        norm(root.left, mx, my, mz, mr, Mx, My, Mz, Mr)
        norm(root.right, mx, my, mz, mr, Mx, My, Mz, Mr)
        return 

def normalize_features(root):
    features = []
    features = traversefeatures(root, features)
    
    x = [tensor[0] for tensor in features]
    y = [tensor[1] for tensor in features]
    z = [tensor[2] for tensor in features]
    r = [tensor[3] for tensor in features]
 
    norm(root, min(x), min(y), min(z), min(r), max(x), max(y), max(z), max(r))

    return 

def traversefeatures(root, features):
       
    if root is not None:
        traversefeatures(root.left, features)
        features.append(root.radius)
        traversefeatures(root.right, features)
        return features

def calcularLossAtributo(nodo, radio):
    if nodo is None:
        return
    radio = radio.reshape(-1,4)
    nodo = nodo.radius.reshape(-1,4)
    l2    = nn.MSELoss()
   
    mse = l2(radio, nodo)
    return mse

def searchNode(node, key):
     
    if (node == None):
        return False
 
    if (node.data == key):
        return node
        
 
    """ then recur on left subtree """
    res1 = searchNode(node.left, key)
    # node found, no need to look further
    if res1:
        return res1
 
    """ node is not found in left,
    so recur on right subtree """
    res2 = searchNode(node.right, key)
    return res2

def getLevelUtil(node, data, level):
    if (node == None):
        return 0
 
    if (node.data == data):
        return level
 
    downlevel = getLevelUtil(node.left, data, level + 1)

    if (downlevel != 0):
        return downlevel
 
    downlevel = getLevelUtil(node.right, data, level + 1)
    return downlevel
 
# Returns level of given data value
 
 
def getLevel(node, data):
    return getLevelUtil(node, data, 1)
 

def set_tree_level(data_loader):
    for d in data_loader:
        for data in d:
            count = []
            numerar_nodos(data, count)
            c = []
            n_nodes = data.count_nodes(data, c)
            for x in range(0, len(n_nodes)):
                level = getLevel(data, x)
                if (level):
                    node = searchNode(data, x)
                    node.level = getLevel(data, x)
                else:
                    print(x, "is not present in tree")
            tree_level = []
            data.get_tree_level(data, tree_level)
            data.set_tree_level(data, sum(tree_level))

def calcularLossEstructura(cl_p, original, mult):
        
        if original is None:
            return
        ce = nn.CrossEntropyLoss(weight = mult)

        if original.childs() == 0:
            vector = [1, 0, 0] 
        if original.childs() == 1:
            vector = [0, 1, 0]
        if original.childs() == 2:
            vector = [0, 0, 1] 


        c = ce(cl_p, torch.tensor(vector, device=device, dtype = torch.float).reshape(1, 3))
        return c

'''
def number_nodes(data_loader, batch_size):
    n_no = []
    qzero = 0
    qOne = 0
    qtwo = 0
    for batch in data_loader:
        for tree in batch:
            count = []
            n = tree.count_nodes(tree, count)
            n_no.append(len(n))
            li = []
            tree.traverseInorderChilds(tree, li)
            zero = [a for a in li if a == 0]
            one = [a for a in li if a == 1]
            two = [a for a in li if a == 2]
            qzero += len(zero)
            qOne += len(one)
            qtwo += len(two)

    nprom = np.mean(n_no)
    qzero /= len(data_loader)*batch_size
    qOne /= len(data_loader)*batch_size
    qtwo /= len(data_loader)*batch_size
    if round(qzero) == 0:
        qzero = 1
    if round(qOne) == 0:
        qOne = 1
    if round(qtwo) == 0:
        qtwo = 1
    mult = torch.tensor([1/round(qzero),1/round(qOne),1/round(qtwo)], device = device)
    return mult
    '''

'\ndef number_nodes(data_loader, batch_size):\n    n_no = []\n    qzero = 0\n    qOne = 0\n    qtwo = 0\n    for batch in data_loader:\n        for tree in batch:\n            count = []\n            n = tree.count_nodes(tree, count)\n            n_no.append(len(n))\n            li = []\n            tree.traverseInorderChilds(tree, li)\n            zero = [a for a in li if a == 0]\n            one = [a for a in li if a == 1]\n            two = [a for a in li if a == 2]\n            qzero += len(zero)\n            qOne += len(one)\n            qtwo += len(two)\n\n    nprom = np.mean(n_no)\n    qzero /= len(data_loader)*batch_size\n    qOne /= len(data_loader)*batch_size\n    qtwo /= len(data_loader)*batch_size\n    if round(qzero) == 0:\n        qzero = 1\n    if round(qOne) == 0:\n        qOne = 1\n    if round(qtwo) == 0:\n        qtwo = 1\n    mult = torch.tensor([1/round(qzero),1/round(qOne),1/round(qtwo)], device = device)\n    return mult\n    '

In [6]:
def my_collate(batch):
    return batch


class tDataset(Dataset):
    def __init__(self, l, dir, transform=None):
        self.names = l
        self.transform = transform
        self.data = [] #lista con las strings de todos los arboles
        for file in self.names:
            self.data.append(read_tree(file, dir))
        self.trees = []
        for tree in self.data:
            deserial = deserialize(tree)
            #normalize_features(deserial)
            self.trees.append(deserial)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        #file = self.names[idx]
        #string = read_tree(file)
        tree = self.trees[idx]
        return tree

batch_size = 1

In [7]:
def decode_testing(v, max, decoder, mult, min):
    def decode_node(v, max, decoder, mult, min, nbif):

        cl = decoder.nodeClassifier(v)
        _, label = torch.max(cl, 1)
        label = label.data
        
        
        if label == 1 and createNode.count <= max:
            #print("label 1")
            right, radius = decoder.internalDecoder(v)
                
            d = createNode(1, radius) 
            
            d.right = decode_node(right, max, decoder, mult, min, nbif = nbif)
            return d

        elif label == 2 and createNode.count <= max:
            #print("label 2")
            left, right, radius = decoder.bifurcationDecoder(v)
                
            d = createNode(1, radius)
            
            d.right = decode_node(right, max, decoder, mult, min, nbif = nbif+1)
            d.left = decode_node(left, max, decoder, mult, min, nbif = nbif+1)
        
            return d

        elif label == 0 : ##output del classifier
            #print("label 0")
            if nbif >= 2:
                #print("mayor que min")
                radio = decoder.featureDecoder(v)
                return createNode(1,radio)
        
            else:
                #print("menor que min")
                left, right, radius = decoder.bifurcationDecoder(v)
                d = createNode(1, radius)
                d.right = decode_node(right, max, decoder, mult, min, nbif = nbif+1)
                d.left = decode_node(left, max, decoder, mult, min, nbif = nbif+1)
                return d

        '''
        elif label == 0 : ##output del classifier
            print("0", createNode.count)
            radio = decoder.featureDecoder(v)
            return createNode(1,radio)  
        '''

    createNode.count = 0
    dec = decode_node (v, max, decoder, mult, min, nbif = 0)

    return dec

In [8]:
def decode_testing2(v, max, decoder, mult):
    def decode_node(v, max, decoder, mult):
        
        cl = decoder.nodeClassifier(v)
        _, label = torch.max(cl, 1)
        label = label.data
        
        if label == 0 and createNode.count <= max: ##output del classifier
           
            radio = decoder.featureDecoder(v)
            
            return createNode(1,radio)

        elif label == 1 and createNode.count <= max:
       
            right, radius = decoder.internalDecoder(v)
                
            d = createNode(1, radius) 
            
            d.right = decode_node(right, max, decoder, mult)
            

            return d
       
        elif label == 2 and createNode.count <= max:
            left, right, radius = decoder.bifurcationDecoder(v)
                
            d = createNode(1, radius)
            
            d.right = decode_node(right, max, decoder, mult)
            d.left = decode_node(left, max, decoder, mult)
            
           
            return d

    createNode.count = 0
    v = decoder.sample_decoder(v)
    dec = decode_node (v, max, decoder, mult)

    return dec

In [9]:
def distance(node1, node2):
    """
    Calculates the Euclidean distance between two nodes
    """
    if node1 is not None and node2 is not None:
        dist = ((node1.radius[0]-node2.radius[0])**2 + 
                (node1.radius[1]-node2.radius[1])**2 + 
                (node1.radius[2]-node2.radius[2])**2)**0.5
    else:
        dist = 0
    return dist

def total_length(root, node, d):
    '''
    calcula la distancia del primer a ultimo nodo, arbol sin bifurcaciones
    '''
    if node.left:
        total_length(root, node.left, d)
    if node.isLeaf():
        #d = distance(root, node)
        d.append(distance(root, node))
        return 
    if node.right:
        total_length(root, node.right, d)

def total_path_length(node):
    """
    Calculates the total path length of the tree
    """
    if node is None:
        return 0
    else:
        return distance(node, node.left) + total_path_length(node.left) + distance(node, node.right) + total_path_length(node.right)

def tortuosity(root):
    """
    Calculates the tortuosity of the tree
    """
    total_path_length_value = total_path_length(root)
    d = []
    max_path_length_value = total_length(root,root, d)
    return total_path_length_value / d[0]

Test random

In [10]:
##Decoder
a = [1.,1.,1.]
mult = torch.Tensor(a)
latent_size = 64
Grassdecoder = GRASSDecoder(latent_size=latent_size, hidden_size=256, mult = mult.to(device))
Grassdecoder = Grassdecoder.to(device)
Grassdecoder.eval()

checkpoint = torch.load("output/AneuriskP15eps02-best-nuevoweight.pth")
#checkpoint = torch.load("outputs/modelo-best-nueva.pth")
#checkpoint = torch.load("outputs/10arboles.pth")
Grassdecoder.load_state_dict(checkpoint['decoder_state_dict'])
epoch = checkpoint['epoch']
print("epoch", epoch)
 

i = 0
d = None
n = 1
radios = []
with torch.no_grad():
    for i in range(2*n):
        noise = torch.randn(batch_size, latent_size).to(device)
        generated_images = decode_testing2(noise, 100, Grassdecoder, mult)
        count = []
        numerar_nodos(generated_images, count)
        #generated_images.traverseInorder(generated_images)
        if d is None:
            d = sTree(generated_images, True, c = "black", s = [n,2,i])
        else:
            sTree(generated_images, True, c = "black", s = [n,2,i], d=d)
          

        i += 1

epoch 19957


In [11]:
p

NameError: name 'p' is not defined

test con input

In [ ]:
#1-2-4
#t_list = os.listdir("./arboles/p5n/" )[:2] #+ os.listdir("./arboles/p2v/" )[2:3]
t_list = os.listdir("resample_Eps02/p10/" )[:5]
dataset = tDataset(t_list, "resample_Eps02/p10/")
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)
a = [1.,1.,1.]
mult = torch.Tensor(a)

#checkpoint = torch.load("outputs/server/64.pth")
checkpoint = torch.load("outputs/modelo-best-nueva.pth")
#checkpoint = torch.load("outputs/10arboles.pth")
#set_tree_level(data_loader)
latent_size = 16
feature_size = latent_size 
##Encoder
Grassencoder = GRASSEncoder(input_size = 4, feature_size=feature_size, hidden_size=512)
Grassencoder = Grassencoder.to(device)

##Decoder
Grassdecoder = GRASSDecoder(latent_size=latent_size, hidden_size=256, mult = mult.to(device))
Grassdecoder = Grassdecoder.to(device)

Grassencoder.load_state_dict(checkpoint['encoder_state_dict'])
Grassdecoder.load_state_dict(checkpoint['decoder_state_dict'])
epoch = checkpoint['epoch']
#loss = checkpoint['loss']
print("epoch", epoch)
Grassencoder.eval()
Grassdecoder.eval()

i = 0
d = None
encodeados = []
for batch in data_loader:
    for input in batch:
        enc_fold = torch_f.Fold(device)
        enc_fold_nodes = []
        enc_fold_nodes.append(encode_structure_fold(enc_fold, input))
        enc_fold_nodes = enc_fold.apply(Grassencoder, [enc_fold_nodes])
        encoded = enc_fold_nodes[0]
        #encoded, kld = torch.chunk(encoded, 2, 1)
        encoded = torch.split(encoded, latent_size, 1)
        encodeados.append(encoded)
        #print("encoded", encoded[0])
        decoded = decode_testing(encoded[0], 100, Grassdecoder, mult)
        
        count = []
        numerar_nodos(decoded, count)
        c = []
        n_nodes = len(input.countNodes(input,c))

        if d is None:
            d = sTree(input, False, c = "black", s = [10,2,i])
        else:
            sTree(input, False, c = "black", s = [10,2,i], d=d)
            
        #input.traverseInorder(input)
        sTree(decoded, True,  s = [10,2,i+1], c = "black", d=d)
        #print("radius" ,input.radius)
        #graph = nx.Graph()
        #input.toGraph( graph, 0, False)
        #nx.write_gpickle(graph, "./profundidad10/grafos/" +str(i)+".gpickle" )
        i += 2
#d.save("dplot.html")

epoch 33051


In [ ]:
t_list = ['ArteryObjAN175-15_tree.dat', 'ArteryObjAN192-4_tree.dat', 'ArteryObjAN157-11_tree.dat', 'ArteryObjAN172-13_tree.dat', 'ArteryObjAN196-6_tree.dat', 'ArteryObjAN172-15_tree.dat', 
'ArteryObjAN189-2_tree.dat', 'ArteryObjAN174-17_tree.dat', 'ArteryObjAN174-9_tree.dat', 'ArteryObjAN163-7_tree.dat']
#t_list = ['ArteryObjAN175-15_tree.dat', 'ArteryObjAN192-4_tree.dat']
#t_list = ['ArteryObjAN1-7_tree.dat']
dataset = tDataset(t_list)
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)
i=0
d = None
for batch in data_loader:
    for input in batch:
        if d is None:
            #d = sTree(input, False, c = "red", s = [2,2,i])
            d = sTree(input, False, c = "red", s = [10,10,i])
        else:
            sTree(input, False, c = "red", s = [10,10,1], d=d)
            #sTree(input, False, c = "red", s = [2,2,1], d=d)

        i += 1
#d.save("data.html")

TypeError: tDataset.__init__() missing 1 required positional argument: 'dir'

In [ ]:
plotTree(generated_images, True)

: 

In [ ]:
print(generated_images.right.radius)
generated_images.radius
#print(generated_images.left.radius)
#print(generated_images.left.right.radius)


: 

In [ ]:
plotTree(input, False)

: 

In [ ]:
'''
def plotTree( root, dec ):
    graph = nx.Graph()
    root.toGraph( graph, 0, dec)
    edges=nx.get_edge_attributes(graph,'procesada')
   
    p = mp.plot( np.array([ graph.nodes[v]['posicion'] for v in graph.nodes]), shading={'point_size':0.1}, return_plot=True)
    print("nodes", graph.nodes)
    for arista in graph.edges:
        p.add_lines( graph.nodes[arista[0]]['posicion'], graph.nodes[arista[1]]['posicion'])

    return 
    '''

: 

In [ ]:
def getCountAtLevel (node, curr, desired, n):
    # If this node doesn't exist, must be zero.
    if node is None: 
        return 0

    # If this node is at desired level, must be one.
    if curr == desired: 
        n.append(node)
        return n

    # Otherwise sum of nodes at that level in left and right sub-trees.
    getCountAtLevel (node.left,  curr+1, desired, n)
    getCountAtLevel (node.right, curr+1, desired, n)
    return n

def cr(root):
    h = root.height(root)
    for i in range(0, h): #i es el nivel del nodo
        n = []
        #print("nodos en nivel ", i, getCountAtLevel(root, 0, i, n))
        c = getCountAtLevel(root, 0, i, n)
        #print("level", i)
        for j in range(len(c)):# j es el j esimo nodo en ese nivel
            #c[j].radius = [i*3.+j+i,i*3.-j,i*1.,i*1.]
            #c[j].radius = [i*3.+j,i*1.+j,i*1.+j,i*1.]
            x = j+1.
            y = i+1.
            z = 1.
            r = 1.
            #print("i,j", i, j)
            #print("radius", x,y,z,r)
            c[j].radius = torch.Tensor([x,y,z,r])
            #c[j].radius = [x,y,z,r]

: 

In [ ]:
cr(input)
#input.traverseInorder(input)
graph = nx.Graph()
input.toGraph( graph, 0, False)

: 

In [ ]:
input = Node(1, radius = torch.tensor((1,1,1,0.5), device = device))
input.right = Node (2,radius = torch.tensor((1.5, 2, 2, 0.5), device = device))
input.right.right = Node(3, radius = torch.tensor((2,2.5,3,0.5), device = device))

input.right.right.right = Node(4, radius = torch.tensor((2.5,3,3.5,0.5), device = device))
input.right.right.right.right = Node(6, radius = torch.tensor((3,3.2,3.5,0.5), device = device))
input.right.right.right.right = Node(10, radius = torch.tensor((3.3,3.2,3.5,0.5), device = device))

input.right.right.left = Node(5, radius = torch.tensor((2.5,2.4,2,0.5), device = device))
input.right.right.left.right = Node(7, radius = torch.tensor((2.7,2.3,2,0.5), device = device))
input.right.right.left.right.right = Node(8, radius = torch.tensor((3.2,2.3,1.9,0.5), device = device))
input.right.right.left.right.right.right = Node(9, radius = torch.tensor((3.6,2.3,1.9,0.5), device = device))

: 

In [ ]:
t_list = os.listdir("./Trees/" )[4:5]
#t_list = ['ArteryObjAN1-7_tree.dat']
print(t_list)
dataset = tDataset(t_list, 'Trees')
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)
input = iter(data_loader).next()
input
input = input[0]
cr(input)

: 

In [ ]:
plotTree(input, False)

: 

In [ ]:
plotTree(input, False)

: 

In [ ]:
graph = nx.Graph()
input[0].toGraph( graph, 0, False)
nx.write_gpickle(graph, "grafo-overview.gpickle" )

: 

In [ ]:
i = 0
d = None
n = 5
for batch in data_loader:
    for input in batch:
        '''
        if d is None:
            d = sTree(input, False, c = "red", s = [20,2,i])
        else:
            sTree(input, False, c = "red", s = [20,2,i], d=d)
            '''
        cr(input)
        print(input.radius)
        #sTree(input, False,  s = [20,2,i+1], c = "black", d=d)
        serial = input.serialize(input)
        file = open("./rectas/" + str(i) +"-nl.dat", "w")
        file.write(serial)
        file.close() 
        i += 2

: 

: 

In [ ]:
t_list = os.listdir("./Trees/")[50:51]
dataset = tDataset(t_list)
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)


for batch in data_loader:
    for input in batch:
        print("input, height",t_list, input.height(input))

: 

In [ ]:
t_list = os.listdir("./Trees/")[67:68]
#t_list = ['ArteryObjAN1-7_tree.dat']
dataset = tDataset(t_list)
data_loader = DataLoader(dataset, batch_size = batch_size, shuffle=True, collate_fn=my_collate)


for batch in data_loader:
    for input in batch:
        print("input, height",t_list, input.height(input))

: 

: 